# **Import Libraries**


In [ ]:
import numpy as np
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Conv2D, Dense, Flatten, MaxPool2D
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping


from keras.saving import hdf5_format
from keras.saving import saving_utils
from keras.saving.saved_model import load as saved_model_load
from keras.saving.saved_model import load_context
from keras.saving.saved_model import save as saved_model_save

# **Import Zip file from local device and unzip the file**

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
#unzip the training data

from zipfile import ZipFile
file_name = "/content/training_set.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

In [ ]:
#unzip the test data

from zipfile import ZipFile
file_name1 = "/content/test_set.zip"

with ZipFile(file_name1, 'r') as zip1:
  zip1.extractall()
  print('Done')

# **Train data and Test data**

In [ ]:
trdata=ImageDataGenerator()
traindata=trdata.flow_from_directory(directory="/content/drive/MyDrive/DL course/training_set", 
target_size=(224,224))

Found 8005 images belonging to 2 classes.


In [ ]:
tsdata=ImageDataGenerator()
testdata=tsdata.flow_from_directory(directory="/content/drive/MyDrive/DL course/test_set", 
target_size=(224,224))

Found 2023 images belonging to 2 classes.


# **Model Generation**

**Sequential Layer**

In [ ]:
model= Sequential()

**Convolution layer and Max pooling layer**

In [ ]:
#input layer
model.add(Conv2D(input_shape=(224,224,3),filters=64, kernel_size=(3,3),
                 padding='same',activation='relu')) 
#Hidden layer
model.add(Conv2D(filters=64, kernel_size=(3,3),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

In [ ]:
model.add(Conv2D(filters=128, kernel_size=(3,3),padding='same',activation='relu'))
model.add(Conv2D(filters=128, kernel_size=(3,3),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

In [ ]:
model.add(Conv2D(filters=256, kernel_size=(3,3),padding='same',activation='relu'))
model.add(Conv2D(filters=256, kernel_size=(3,3),padding='same',activation='relu'))
model.add(Conv2D(filters=256, kernel_size=(3,3),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

In [ ]:
model.add(Conv2D(filters=512, kernel_size=(3,3),padding='same',activation='relu'))
model.add(Conv2D(filters=512, kernel_size=(3,3),padding='same',activation='relu'))
model.add(Conv2D(filters=512, kernel_size=(3,3),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

In [ ]:
model.add(Conv2D(filters=512, kernel_size=(3,3),padding='same',activation='relu'))
model.add(Conv2D(filters=512, kernel_size=(3,3),padding='same',activation='relu'))
model.add(Conv2D(filters=512, kernel_size=(3,3),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

**Output Layer**

In [ ]:
model.add(Flatten())
model.add(Dense(units=4096,activation='relu')) #fully connected layer
model.add(Dense(units=4096,activation='relu'))
model.add(Dense(units=2,activation='softmax')) # number of output class is 2

# **Model Compilation**

In [ ]:
#optimizer, loss function and accuracy metrics
opt=Adam(learning_rate=.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                    

In [ ]:
checkpoint=ModelCheckpoint("/content/drive/MyDrive/DL course/ModelFiles", monitor='val_acc',
                           verbose=1, save_best_only=True,
                           save_weights_only=False, mode='auto',
                           save_freq=1)

In [ ]:
earlystop= EarlyStopping(monitor='val_acc', min_delta=0, patience=20, 
                         verbose=1,mode='auto')

# **Model Fitting**

In [ ]:
hist=model.fit_generator(steps_per_epoch=10, generator=traindata,
                         validation_data=testdata, validation_steps=10,
                         epochs=5,callbacks=[checkpoint,earlystop])

<ipython-input-16-a09e97de3b02>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist=model.fit_generator(steps_per_epoch=10, generator=traindata,


Epoch 1/5


 1/10 [==>...........................] - ETA: 11:43 - loss: 0.6932 - accuracy: 0.5000

 2/10 [=====>........................] - ETA: 7:47 - loss: 0.6923 - accuracy: 0.5469 

 3/10 [========>.....................] - ETA: 6:40 - loss: 0.6918 - accuracy: 0.5729

...

 7/10 [====================>.........] - ETA: 2:50 - loss: 0.6983 - accuracy: 0.5045

 8/10 [=======================>......] - ETA: 1:54 - loss: 0.6991 - accuracy: 0.5000

 9/10 [==========================>...] - ETA: 57s - loss: 0.6979 - accuracy: 0.5104 

10/10 [==============================] - ETA: 0s - loss: 0.6974 - accuracy: 0.5125 

10/10 [==============================] - 729s 74s/step - loss: 0.6974 - accuracy: 0.5125 - val_loss: 0.6961 - val_accuracy: 0.4719


# **Prediction of test data**

In [ ]:
img=tf.keras.utils.load_img("/content/test_set/cats/cat.4012.jpg",target_size=(224,224))
img=np.asarray(img)
plt.imshow(img)


In [ ]:
img=np.expand_dims(img,axis=0)
saved_model=load_model("vgg16_1.h5")
output =saved_model.predict(img)
if output[0][0]>output[0][1]:
  print("cat")
else:
  print("dog")